In [3]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity

In [4]:
raw_data = pd.read_csv("./data/data_selection_17_23.csv")

In [5]:
raw_data

,timestamp,nodeid,temperature,humidity,voltage,light
0,1468800,1,NaN,NaN,NaN,NaN
1,1468800,2,NaN,NaN,NaN,NaN
2,1468800,3,NaN,NaN,NaN,NaN
3,1468800,4,NaN,NaN,NaN,NaN
4,1468800,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
933115,1987170,50,NaN,NaN,NaN,NaN
933116,1987170,51,NaN,NaN,NaN,NaN
933117,1987170,52,NaN,NaN,NaN,NaN
933118,1987170,53,NaN,NaN,NaN,NaN


In [6]:
non_nan_data = raw_data.dropna()

In [7]:
non_nan_data

,timestamp,nodeid,temperature,humidity,voltage,light
65178,1505010,1,21.8112,34.64030,2.48502,478.40
65179,1505010,2,22.3306,35.23020,2.46440,537.28
65180,1505010,3,21.8896,35.02220,2.49546,412.16
65187,1505010,10,22.3600,34.32730,2.49546,625.60
65190,1505010,13,19.9296,39.00820,2.44412,1494.08
...,...,...,...,...,...,...
931613,1986360,6,122.1530,-3.91901,2.22442,0.92
931761,1986420,46,76.9362,45.37430,2.21612,1.38
931849,1986480,26,19.8904,53.12660,2.32910,0.46
931935,1986540,4,122.1530,38.63340,2.29313,5.06


In [8]:

def detect_anomalies_kde(data, column):
    original_index = data.index
    reshaped_data = data[column].values.reshape(-1, 1)

    # Perform Kernel Density Estimation
    kde = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(reshaped_data)
    log_density = kde.score_samples(reshaped_data)
    threshold = np.percentile(log_density, 1)  # Use 1% as the threshold
    anomaly_indices = original_index[log_density < threshold]

    return data.loc[anomaly_indices]

In [9]:
anomalous_temperature = detect_anomalies_kde(non_nan_data, 'temperature')
anomalous_humidity = detect_anomalies_kde(non_nan_data, 'humidity')
anomalous_voltage = detect_anomalies_kde(non_nan_data, 'voltage')
anomalous_light = detect_anomalies_kde(non_nan_data, 'light')

In [10]:
print(len(anomalous_temperature), "temperature anomalies")
print(len(anomalous_humidity), "humidity anomalies")
print(len(anomalous_voltage), "voltage anomalies")
print(len(anomalous_light), "light anomalies")

1844 temperature anomalies
1838 humidity anomalies
1622 voltage anomalies
1812 light anomalies


In [11]:
# save the results
anomalous_temperature.to_csv("./data/anomalous_temperature.csv", index=False)
anomalous_humidity.to_csv("./data/anomalous_humidity.csv", index=False)
anomalous_voltage.to_csv("./data/anomalous_voltage.csv", index=False)
anomalous_light.to_csv("./data/anomalous_light.csv", index=False)


In [ ]:
# Check and print anomalies for each attribute separately
print("Anomalous sensors in temperature:")
print(set(anomalous_temperature['nodeid']))

print("Anomalous sensors in humidity:")
print(set(anomalous_humidity['nodeid']))

#print("Anomalous sensors in voltage:")
#print(set(anomalous_voltage['nodeid']))

print("Anomalous sensors in light:")
print(set(anomalous_light['nodeid']))


Anomalous sensors in temperature:
{32, 35, 4, 6, 38, 39, 43, 13, 46, 19, 21, 24, 26, 29}
Anomalous sensors in humidity:
{32, 34, 35, 36, 6, 39, 43, 13, 47, 16, 17, 19, 25, 26, 27, 29, 30}
Anomalous sensors in light:
{1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 14, 16, 17, 19, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54}


In [ ]:
# Find the nodeid intersection of anomalies across all measurements
anomalous_sensors = set(anomalous_temperature['nodeid']) & \
                    set(anomalous_humidity['nodeid']) & \
                    set(anomalous_light['nodeid']) #set(anomalous_voltage['nodeid']) & \

print("Sensors that are anomalous across all attributes:")
print(anomalous_sensors)

Sensors that are anomalous across all attributes:
{32, 35, 6, 39, 43, 13, 19, 26, 29}


In [ ]:
anomalous_temperature

,timestamp,nodeid,temperature,humidity,voltage,light
114086,1532160,39,90.9208,137.5120,0.009101,9.206940e-11
435760,1710870,35,38.0400,44.8460,2.320000,1.729600e+02
435976,1710990,35,38.3830,44.9782,2.320000,1.729600e+02
436084,1711050,35,38.3634,44.9782,2.320000,1.729600e+02
436192,1711110,35,38.2556,44.9782,2.320000,1.729600e+02
...,...,...,...,...,...,...
917721,1978620,46,72.0754,44.3162,2.224420,1.380000e+00
920907,1980390,46,74.2510,44.3494,2.224420,1.380000e+00
922851,1981470,46,74.2314,44.5151,2.224420,1.380000e+00
926069,1983270,24,63.3338,52.3659,2.338270,9.200000e-01


In [ ]:
anomalous_humidity

,timestamp,nodeid,temperature,humidity,voltage,light
69036,1507140,25,30.0922,22.17190,2.56000,1847.36
77136,1511640,25,29.5630,22.31870,2.56000,1847.36
79026,1512690,25,29.4160,22.39200,2.56000,1847.36
82150,1514430,17,122.1530,22.97800,2.29313,1140.80
82312,1514520,17,122.1530,20.69930,2.29313,1023.04
...,...,...,...,...,...,...
794860,1910370,35,122.1530,-2.70687,2.23278,0.92
795076,1910490,35,122.1530,-3.43355,2.22442,0.92
795130,1910520,35,122.1530,-3.27191,2.23278,0.92
795346,1910640,35,122.1530,-3.27191,2.23278,0.92


In [ ]:
anomalous_light

,timestamp,nodeid,temperature,humidity,voltage,light
111840,1530930,7,22.2914,35.7148,2.48502,28.52
112056,1531050,7,22.3600,35.5764,2.48502,24.84
112415,1531230,42,24.4180,31.7016,2.49546,28.52
112469,1531260,42,24.4278,31.7368,2.49546,28.52
112492,1531290,11,21.9288,38.1213,2.43410,34.04
...,...,...,...,...,...,...
899623,1968570,38,122.1530,44.5151,2.27556,19.32
900001,1968780,38,122.1530,44.5813,2.28431,19.32
900055,1968810,38,122.1530,44.5482,2.28431,19.32
900217,1968900,38,122.1530,44.5813,2.28431,19.32
